**Import requiered libraries**

In [24]:
from huggingface_hub import login
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import wandb
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

**Load data from Hugging face**

In [3]:
secret_label = "secret_val_0"
secret_value = UserSecretsClient().get_secret(secret_label)
login(secret_value)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
dataset = load_dataset("OmidAghili/Sentiment_Analysis_Train-Validation")
testDataset = load_dataset("OmidAghili/Sentiment_Analysis_Test")

Generating train split:   0%|          | 0/209519 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/171426 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['reviewText', 'label'],
        num_rows: 209519
    })
    validation: Dataset({
        features: ['reviewText', 'label'],
        num_rows: 171426
    })
})

In [6]:
testDataset

DatasetDict({
    train: Dataset({
        features: ['reviewText'],
        num_rows: 20000
    })
})

In [7]:
dataset['train'][0]

{'reviewText': 'Too early to judge but so far it has been good; it is very practical, light in my hands, easy plug and play, I bought four and plugged them on four Machins they all work fine , lets see how long they last, other pluged mouses i have worked for eight years.',
 'label': 3}

In [8]:
testDataset['train'][0]

{'reviewText': "The name and description of this device are misleading.  It is supposed to be a conversion tray for a laptop optical drive.  I assumed that you would be able to remove the old drive and replace it with this new card.  However, this is far from the case.  The engineers who created this device either use different laptops than the rest of us, or they designed it for a desktop.  There is no way that I am going to pull apart my laptop to connect this device.  I don't even have the proper ports on my laptop's motherboard.\n\nI'm a computer savvy guy.  I've repaired many laptops and other devices over the years, so if there were some easy modification that I could do to this device to make it work, then I would do it.  However, that is not the case.\n\nFor the life of me, I don't know what these folks were thinking when they designed this.  If I wanted to install USB 3 on a desktop, I wouldn't do it through the optical drive!  I would get a dedicated card for that.\n\nI'm dis

**Model**

In [25]:
wandb.init(project="sentiment_analysis")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [26]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["reviewText"], truncation=True)

tokenized = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")


Map:   0%|          | 0/171426 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1) 
    accuracy = accuracy_score(labels, predictions)
    wandb.log({"accuracy": accuracy})
    return {"accuracy": accuracy}
        
#     return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "EXTREMELY NEGATIVE", 1: "NEGATIVE",2:"NEURAL",3:"POSITIVE", 4:"EXTREMELY POSITIVE"}
label2id = {"EXTREMELY NEGATIVE": 0, "NEGATIVE": 1,"NEURAL":2,"POSITIVE":3, "EXTREMELY POSITIVE":4}


model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=5, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=2e-5,
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="wandb" 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [ ]:
trainer.push_to_hub("OmidAghili/Amazon_Reviews")